In [10]:
import tensorflow
import pandas as pd
import numpy as np
from tensorflow import keras
from collections import Counter
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense


Using TensorFlow backend.


In [18]:
def convert(arr):
    array = []
    for i,item in enumerate(arr):
        if item > 0.5:
            array.append(True)
        else:
            array.append(False)
    return array

In [11]:
train_data = pd.read_csv("full_training_data_1988-2016")
test_data = pd.read_csv("testing_data_2017-2020")

x_train = train_data.iloc[:,1:-1].select_dtypes(exclude=['object'])
y_train = train_data["All_NBA"]
x_test = test_data.iloc[:,1:-1].select_dtypes(exclude=['object'])
y_test = test_data["All_NBA"]

x_train_numpy = x_train.to_numpy()
y_train_numpy = y_train.to_numpy()

x_test_numpy = x_test.to_numpy()
y_test_numpy = y_test.to_numpy()

print(x_train.shape[1])



90


In [28]:

neural_model = Sequential([
     Dense(units=90, input_dim=90, activation='relu'),
     Dense(units=50,activation='relu'),
     Dropout(0.2),
     Dense(32,activation='relu'),
     Dense(16,activation='relu'),
     Dropout(0.2),
     Dense(8,activation='relu'),
     Dropout(0.2),
     Dense(4,activation='relu'),
     Dense(1,activation='sigmoid')  
    
    
])

neural_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 90)                8190      
_________________________________________________________________
dense_7 (Dense)              (None, 50)                4550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                1632      
_________________________________________________________________
dense_9 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 8)                

In [29]:
counts = np.bincount(y_train_numpy)

all_nba_weight = 1.0/counts[0]
non_all_nba_weight = 1.0/counts[1]

metrics = [
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
]

neural_model.compile(
    optimizer = keras.optimizers.Adam(),loss="binary_crossentropy",metrics=metrics
)

class_weight = {0: non_all_nba_weight, 1: all_nba_weight}

callbacks = [keras.callbacks.EarlyStopping(patience=10,restore_best_weights = True)]

neural_model.fit(
    x_train_numpy,
    y_train_numpy,
    epochs=150,
    validation_split=0.1,
    verbose=1,
    # callbacks = callbacks,
    class_weight = class_weight
)



4203e-04 - val_fn: 2014.8462 - val_fp: 1929.0000 - val_tn: 56426.4609 - val_tp: 68.0000 - val_precision: 0.0341 - val_recall: 0.0326
Epoch 6/150
10983/10983 [==============================] - 1s 56us/step - loss: 4.9029e-04 - fn: 2225.5872 - fp: 1929.0000 - tn: 62317.3672 - tp: 68.0000 - precision: 0.0341 - recall: 0.0297 - val_loss: 4.4541e-04 - val_fn: 2434.8462 - val_fp: 1929.0000 - val_tn: 68210.4609 - val_tp: 68.0000 - val_precision: 0.0341 - val_recall: 0.0272
Epoch 7/150
10983/10983 [==============================] - 1s 56us/step - loss: 4.0465e-04 - fn: 2643.4448 - fp: 1929.0000 - tn: 74103.4844 - tp: 68.0000 - precision: 0.0341 - recall: 0.0251 - val_loss: 3.6909e-04 - val_fn: 2854.8462 - val_fp: 1929.0000 - val_tn: 79994.4609 - val_tp: 68.0000 - val_precision: 0.0341 - val_recall: 0.0233
Epoch 8/150
10983/10983 [==============================] - 1s 55us/step - loss: 3.3674e-04 - fn: 3061.6221 - fp: 1929.0000 - tn: 85889.3203 - tp: 68.0000 - precision: 0.0341 - recall: 0.0218 

In [30]:
#predicting classes
predicted_y = neural_model.predict(x_test_numpy)
predicted_bool = convert(predicted_y)
score = neural_model.evaluate(x_test_numpy,y_test_numpy)
print(score)
print(accuracy_score(y_test_numpy,predicted_bool))

x_test_copy = test_data.copy()
x_test_copy["Predicted_All_NBA"] = predicted_bool
print()

print(x_test_copy.loc[x_test_copy.All_NBA == False])
print()
print(x_test_copy.loc[x_test_copy.Predicted_All_NBA == True])
print()
#players predicted to be on All-NBA and were not on All-NBA
print(x_test_copy.loc[(x_test_copy.All_NBA == False) & (x_test_copy.Predicted_All_NBA == True),"Player"])
print()
#players predicted not to be on All-NBA and was listed on All-NBA
print(x_test_copy.loc[(x_test_copy.All_NBA == True) & (x_test_copy.Predicted_All_NBA == False),"Player"])
print()


2085/2085 [==============================] - 0s 24us/step
[0.1928380664852861, 62964.15234375, 1930.0, 1766709.375, 68.0, 0.03403402119874954, 0.001078814617358148]
0.9712230215827338

      Unnamed: 0         Player Pos  Age   Tm   G    MP  FG_per_game  \
0              0   Álex Abrines  SG   23  OKC  68  15.5          2.0   
1              1     Quincy Acy  PF   26  TOT  38  14.7          1.8   
2              2   Steven Adams   C   23  OKC  80  29.9          4.7   
3              3  Arron Afflalo  SG   31  SAC  61  25.9          3.0   
4              4  Alexis Ajinça   C   28  NOP  39  15.0          2.3   
...          ...            ...  ..  ...  ...  ..   ...          ...   
2080        2080     Trae Young  PG   21  ATL  60  35.3          9.1   
2081        2081    Cody Zeller   C   27  CHO  58  23.1          4.3   
2082        2082   Tyler Zeller   C   30  SAS   2   2.0          0.5   
2083        2083     Ante Žižić   C   23  CLE  22  10.0          1.9   
2084        2084    Ivi